# Kinetic Energy Mean-Transient Decomposition

Decomposing the kinetic energy into time-mean and transient components.

## Theory

For a hydrostatic ocean model, like MOM5, the relevant kinetic energy per unit mass is 

$$ {\rm KE} = \frac{1}{2} (u^2 + v^2).$$

The vertical velocity component, $w$, does not appear in the mechanical energy budget. It is very much subdominant. But more fundamentally, it simply does not appear in the mechanical energy buget for a hydrostatic ocean. 

For a non-steady fluid, we can define the time-averaged kinetic energy as the __total kinetic energy__, TKE

$$ {\rm TKE} = \left< {\rm KE} \right > {\stackrel{\rm{def}}{=}} \frac{1}{T} \int_0^T \frac{1}{2} \left( u^2 + v^2 \right)\,\mathrm{d}t.$$

It is useful to decompose the velocity into time-mean and time-varying components, e.g.,

$$ u = \bar{u} + u'.$$

The __mean kinetic energy__ is the energy associated with the mean flow

$$ {\rm MKE} = \frac{1}{2} \left( \bar{u}^2 + \bar{v}^2 \right) $$

The kinetic energy of the time varying component is the __eddy kinetic energy__, EKE. This quantity can be obtained by 
substracting the velocity means and calculating the kinetic energy of the 
perturbation velocity quantities.

$$ {\rm EKE} =  \overline{ \frac{1}{2} \left[ \left(u - \overline{u}\right)^2 + \left(v - \overline{v}\right)^2 \right] } $$
                                 
MKE and EKE partition the total kinetic energy

$${\rm TKE} = {\rm MKE} + {\rm EKE}.$$


## Calculation


We start by importing some useful packages.

In [1]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

import cosima_cookbook as cc

from cosima_cookbook import distributed as ccd

import glob
import matplotlib.pyplot as plt
import numpy as np
import cmocean as cm
import xarray as xr

from dask.distributed import Client

In [2]:
# from tqdm import tqdm_notebook

Import cartopy to plot maps:

In [3]:
import cartopy.crs as ccrs
import cartopy.feature as feature

land_50m = feature.NaturalEarthFeature('physical', 'land', '50m',
                                        edgecolor='black',
                                        facecolor='gray',
                                        linewidth=0.2)

Start up a dask cluster.

In [4]:
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 28
Total threads: 28,Total memory: 251.20 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:36415,Workers: 28
Dashboard: /proxy/8787/status,Total threads: 28
Started: Just now,Total memory: 251.20 GiB
Comm: tcp://127.0.0.1:44467,Total threads: 1
Dashboard: /proxy/39667/status,Memory: 8.97 GiB
Nanny: tcp://127.0.0.1:40701,


Create a database session and select an experiment. Here we choose an experiment which has daily velocities saved for the Southern Ocean.

In [5]:
session = cc.database.create_session()
expt = '01deg_jra55v13_ryf9091'

While not difficult to write down, this is fairly involved computation since to compute the eddy kinetic energy requires both the velocity and the mean of the velocity components.  Since the dataset is large, we want to avoid loading all of the velocity data into memory at the same time.

To calculate EKE, we need horizontal velocities $u$ and $v$, preferably saved at `1 daily` frequency (or perhaps `5 daily`).

### Example

For example, let's calculate the mean and eddy kinetic energy of this particular model run. Because the computations are very lengthy we will only load 1 month of daily velocity output.

(If you want to do the decomposition with output longer than, e.g., 1 year then we suggest you either convert this to a `.py` script and submit through the queue via `qsub` or figure a way to scale `dask` up to larger `ncpus`.)

**WARNING**
`ocean_daily_3d_u_%.nc` files prior to 2086-01-01 are on a different domain (SH) than those after this date (global). If you query data spanning both periods, the cosima-cookbook will silently concatenate these resulting in a object on the global domain. However, this will cause issues with downstream analysis

In [6]:
start_time = '2086-01-01'

Here we build datasets for the variables `u` and `v`.

In [7]:
u = cc.querying.getvar(
    expt,
    'u',
    session,
    ncfile='ocean_daily_3d_u_%.nc',
    start_time = start_time,
    chunks={"st_ocean": 7, "yu_ocean": 300, "xu_ocean": 400} # multiple of chunking on disk
)
u = u.sel(time=slice('2086-01-01', '2086-01-31'))

v = cc.querying.getvar(
    expt,
    'v',
    session,
    ncfile='ocean_daily_3d_v_%.nc',
    start_time = start_time,
    chunks={"st_ocean": 7, "yu_ocean": 300, "xu_ocean": 400}
)
v = v.sel(time=slice('2086-01-01', '2086-01-31'))

The kinetic energy is 

$$ {\rm KE} = \frac{1}{2} (u^2 + v^2).$$

We construct the following expression:

In [8]:
KE = 0.5*(u**2 + v**2)

You may notice that this line runs instantly. The calculation is not (yet) computed. Rather, `xarray` needs to broadcast the squares of the velocity fields together to determine the final shape of KE. 

This is too large to store locally.  We need to reduce the data in some way.  

The mean kinetic energy is calculated by this function, which returns the depth integrated KE:

$$ \int_{z_0}^{z} \mathrm{KE}\,\mathrm{d}z.$$

In [9]:
dz = np.gradient(KE.st_ocean)[:, np.newaxis, np.newaxis]
KE_dz = KE*dz

TKE = KE_dz.mean('time').sum('st_ocean')

In [10]:
%%time
# This fails using the dask-2022.10.2 environment, but not using dask-2022.11.0
TKE = TKE.compute()

CPU times: user 41.2 s, sys: 5.94 s, total: 47.1 s
Wall time: 1min 52s


## Mean Kinetic Energy

For the mean kinetic energy, we need to average the velocities over time.

$$ {\rm MKE} = \frac{1}{2} \left( \bar{u}^2 + \bar{v}^2 \right). $$

In [11]:
u_mean = u.mean('time')
v_mean = v.mean('time')

In [12]:
%%time
# This fails using the dask-2022.10.2 environment, but not using dask-2022.11.0
MKE = (0.5*(u_mean**2 + v_mean**2)*dz).sum('st_ocean').compute()

CPU times: user 28.2 s, sys: 4.11 s, total: 32.3 s
Wall time: 1min 1s


## Eddy Kinetic Energy

We calculate the transient component of the velocity field and then compute the EKE:


$$ {\rm EKE} =  \overline{ \frac{1}{2} \left[ \left(u - \overline{u}\right)^2 + \left(v - \overline{v}\right)^2 \right] }. $$

In [13]:
u_transient = u - u_mean
v_transient = v - v_mean

In [14]:
%%time
# This fails using the dask-2022.10.2 environment, but not using dask-2022.11.0
EKE = (0.5*(u_transient**2 + v_transient**2)*dz).sum('st_ocean').mean('time').compute()

CPU times: user 44.5 s, sys: 7.94 s, total: 52.4 s
Wall time: 1min 49s
